# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [20]:
import os

from openai import OpenAI
from dotenv import load_dotenv
from IPython.display import Markdown, display

In [10]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

In [11]:
# set up environment
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

openai = OpenAI()

In [14]:
# here is the question; type over this to ask something new
system_prompt = "You are an assistant that answers technical questions and responds with an explanation. \
Where applicable, you provide concrete code blocks for examples, using python. You may also provide links\
to documentation or other online resources."

code_question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [25]:
# Get gpt-4o-mini to answer, with streaming
def ask_gpt(question):
    stream = openai.chat.completions.create(
        model=MODEL_GPT,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": question}
          ],
        stream=True
    )

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''

    return response

In [27]:
display(Markdown(ask_gpt(code_question)))

This line of code is a Python statement that uses `yield from` with a generator expression that creates a set of authors from a collection of book dictionaries. Let's break it down:

1. **Set Comprehension**: `{book.get("author") for book in books if book.get("author")}` is a set comprehension that iterates over the `books` iterable. For each `book` in `books`, it retrieves the `"author"` from the dictionary using `book.get("author")`. The `if book.get("author")` condition ensures that only books with a non-empty author (i.e., instead of `None` or an empty string) are considered. The set comprehension will return a set of unique authors because sets automatically handle duplicates.

2. **`yield from`**: The expression `yield from` is used in a generator to yield all values from an iterable (in this case, the set created by the comprehension). This means that instead of returning a single value or a complete list, it allows the generator that contains this line to yield each author one at a time to the caller.

### Why Use This Code?

- **Efficiency**: Using a set comprehension quickly collects unique authors without having to manually check for duplicates.
- **Iterability**: The use of `yield from` makes it easy to produce a stream of authors without creating a list in memory, which can be beneficial for handling large datasets.
- **Readability**: The code is concise and expresses the intent clearly: to gather authors and yield them one by one.

### Example Code

Here is a complete example to illustrate how this might be used:

```python
def get_authors(books):
    yield from {book.get("author") for book in books if book.get("author")}

# Example usage:
books = [
    {"title": "Book A", "author": "Author 1"},
    {"title": "Book B", "author": "Author 2"},
    {"title": "Book C", "author": "Author 1"},  # Duplicate author
    {"title": "Book D"},  # No author
    {"title": "Book E", "author": "Author 3"}
]

for author in get_authors(books):
    print(author)
```

### Output
```
Author 1
Author 2
Author 3
```

In the example, `get_authors(books)` collects unique authors from the list of book dictionaries and prints them one by one, with duplicates and books without authors omitted.

### Resources
- Python [Set Comprehensions](https://docs.python.org/3/tutorial/datastructures.html#sets)
- Python [Generators](https://docs.python.org/3/tutorial/classes.html#generators)

In [18]:
# Get Llama 3.2 to answer
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

def ask_ollama(question):
    response = ollama_via_openai.chat.completions.create(
        model=MODEL_LLAMA,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": question}
          ],
    )
    
    return response.choices[0].message.content

In [21]:
display(Markdown(ask_ollama(code_question)))

**Code Explanation**

The given code uses a feature of Python called "yield from" which is used to simplify iteration over multiple iterables.

Here's a breakdown:

- `yield from`: This keyword allows you to expand the current iterator into multiple iterators. It can be thought of as a `for` loop that takes an iterable and yields each item in turn, using the next iteration of another iterator if needed.
  
- `{book.get("author") for book in books}`: This is a generator expression, which creates an iterable object by evaluating an expression for each item in an input iterable. In this case, it's generating a sequence of `author` values from dictionaries in the "books" iterable.

The key part is the use of `yield from`, which takes the sequence generated above and yields each value from these sequences.

**How it works**

1. The expression `{book.get("author") for book in books if book.get("author")}` generates a sequence of authors from the given list of dictionaries.
2. The outer iterator is then replaced by `yield from` which expands this into `yield * ...`, yielding each dictionary's `author` value one at a time.

**Example Use Case**

```python
# Sample data
books = [
    {"title": "Book 1", "author": "John Doe"},
    {"title": "Book 2", "author": "Jane Smith"},
    {},
    {"title": "", "author": "Unknown"}
]

authors = yield from {book.get("author") for book in books if book.get("author")}

print(authors) # Output: ['John Doe', 'Jane Smith']
```

In this example, the `yield from` expression is used to extract a sequence of authors from the `books` list. Without the `yield from`, we'd have an iterator that contains another iterable (a dictionary with only the `author` key). The `yield from` replaces it with one where every value is yielded iteratively.

For example, in the first iteration over the `"Book 1"` entry, `book.get("author")` yields `'John Doe'`. Then for `"Jane Smith"`, `"Author = Jane Smith"` is passed to an inner function where author gets extracted.